In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root
import plotly.graph_objects as go

# An unstable periodic orbit in the restricted three-body problem

We consider a reduced three body problem consisting of the motion of a satellite in the framework of the attraction of the moon and the earth. For the purpose of the exercise, we assume that the system earth-moon is in circular rotation at constant speed  in a planar motion with the mass center of gravity located at the origin and that the mass of the satellite $\epsilon$ is small enough compared the mass of the earth $1-\mu$ and the mass of the moon $\mu$ to so that we can neglect its impact on the earth-moon system.  We also assume that the motion of the satellite is governed by the attraction of the two bodies earth and moon through the Newton gravitation law.

The motion of the satellite in the complex plane satisfies the equation: 

$$
\epsilon\, {\mathrm d}_t^2 Y = \frac{\epsilon(1-\mu)}{\vert\vert A-Y\vert\vert^2}\, \frac{A-Y}{\vert\vert A-Y\vert\vert}+ \frac{\epsilon\mu}{\vert\vert B-Y\vert\vert^2}\, \frac{B-Y}{\vert\vert B-Y\vert\vert}.
$$

In order to eliminate the factor $e^{it}$ in $A=-\mu\,e^{it}$ and  $A=(1-\mu)e^{it}$, we introduce the variable $y=e^{-it}\,Y = y_1+i\,y_2$. In this new referential the earth and the moon are motionless. We have $Y=e^{it}y$ and ${\mathrm d}_t^2 Y = -e^{it}y+2\,i\,e^{it}{\mathrm d}_t y+e^{it}{\mathrm d}_t^2 y$ and the equation of motion thus read:

$$
{\mathrm d}_t^2 y + 2\,i\,{\mathrm d}_t y-y= (1-\mu)\,\frac{-\mu-y}{\vert\vert \mu+y\vert\vert^3}+ \mu\, \frac{1-\mu-y}{\vert\vert 1-\mu-y\vert\vert^3}.
$$

Introducing the real and imaginary parts of $y$ and then switching to a first order system of differential equations, we obtain:

$$
\begin{array}{rcl} 
{\mathrm d}_t y_1 & = & y_3, \\ 
{\mathrm d}_t y_2 & = & y_4, \\ 
{\mathrm d}_t y_3 & = & y_1+2\,y_4- (1-\mu)(y_1+\mu)/r_1^3 - \mu (y_1-1+\mu)/r_2^3, \\
{\mathrm d}_t y_4 & = & y_2-2\,y_3- (1-\mu)y_2/r_1^3 - \mu y_2/r_2^3,
\end{array} 
$$

with $r_1=((y_1+\mu)^2+y_2^2)^{1/2}$ and $((y_1-1+\mu)^2+y_2^2)^{1/2}$.

For the initial values, we have chosen:

$$
\begin{aligned}
y_1(0) &= -0.5655899165951338\\
y_2(0) &=  0.601315396569226\\
y_3(0) &= -0.45171183358756384\\
y_4(0) &= 0.23073427996775764.
\end{aligned}
$$

The motion of the satellite should then be almost periodic, with period $T \approx 17.06521656$.

In [ ]:
class three_body_model:

    def __init__(self, mu):
        self.mu = mu

    def fcn(self, t, y) :
        y1,y2,y3,y4 = y
        mu = self.mu
        r1 = np.sqrt((y1+mu)*(y1+mu) + y2*y2)
        r2 = np.sqrt((y1-1+mu)*(y1-1+mu) + y2*y2)
        y1_dot = y3
        y2_dot = y4
        y3_dot = y1 + 2*y4 - (1-mu)*(y1+mu)/(r1*r1*r1) - mu*(y1 - 1 + mu)/(r2*r2*r2)
        y4_dot = y2 - 2*y3 - (1-mu)*y2/(r1*r1*r1) - mu*y2/(r2*r2*r2)
        return (y1_dot, y2_dot, y3_dot, y4_dot)

In [ ]:
# Initialization
yini = (-0.5655899165951338, 0.601315396569226, -0.45171183358756384, 0.23073427996775764)
#yini = (0.994, 0., 0., -2.00158510637908252240537862224)

tini = 0.
tend = 18.

mu = 0.012277471
    
tbm = three_body_model(mu)
fcn = tbm.fcn

## Quasi-exact solution

The quasi-exact solution is obtained by using an explicit Runge-Kutta method of order(4)5 due to Dormand and Prince with stepsize control (RK45) with very fine tolerances.

In [ ]:
tol = 1.e-12
sol = solve_ivp(fcn, (tini, tend), yini, rtol=tol, atol=tol)

fig_y1y2 = go.Figure()
fig_y1y2.add_trace(go.Scatter(x=sol.y[0], y=sol.y[1], name="quasi-exact sol.", showlegend=True))
fig_y1y2.update_layout(title="Arenstorf orbit", xaxis_title="y1", yaxis_title="y2")
fig_y1y2.show()

fig_sol = go.Figure()
fig_sol.add_trace(go.Scatter(x=sol.t, y=sol.y[0], name="y1"))
fig_sol.add_trace(go.Scatter(x=sol.t, y=sol.y[1], name="y2"))
fig_sol.add_trace(go.Scatter(x=sol.t, y=sol.y[2], name="y3"))
fig_sol.add_trace(go.Scatter(x=sol.t, y=sol.y[3], name="y4"))
fig_sol.update_layout(title="Evolution of solutions", xaxis_title="t")
fig_sol.show()

## Forward Euler

In [ ]:
class ode_result:
    def __init__(self, y, t):
        self.y = y
        self.t = t

def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.array(fcn(tn, yn))

    return ode_result(y, t)

In [ ]:
tini = 0.
tend = 18.
nt = 180000
sol_fe = forward_euler(tini, tend, nt, yini, fcn)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sol.y[0], y=sol.y[1], name="quasi-exact sol."))
fig.add_trace(go.Scatter(x=sol_fe.y[0], y=sol_fe.y[1], name="forward Euler"))
fig.update_layout(title="Arenstorf orbit", xaxis_title="y1", yaxis_title="y2")
fig.show()

## Backward Euler

In [ ]:
def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini_array = np.array(yini)
    neq = yini_array.size

    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini_array

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*np.array(fcn(tip1, uip1))

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*np.array(fcn(tn, yn))
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        sol = root(g, y0, (yn, tn+dt))
        y[:,it+1] = sol.x

    return ode_result(y, t)

In [ ]:
tini = 0.
tend = 18.
nt = 180000
sol_be = backward_euler(tini, tend, nt, yini, fcn)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sol.y[0], y=sol.y[1], name="quasi-exact sol."))
fig.add_trace(go.Scatter(x=sol_be.y[0], y=sol_be.y[1], name="backward Euler"))
fig.update_layout(title="Arenstorf orbit", xaxis_title="y1", yaxis_title="y2")
fig.show()